In [1]:
import os
import pandas as pd
import nltk
os.chdir(r'C:\...................')


In [44]:
import geopy.distance
import googleplaces
from googleplaces import GooglePlaces, types, lang
import pandas as pd
#import numpy as np

API_KEY = '................'
google_places = GooglePlaces(API_KEY)

def queryGooglePlace(lat, lng, radius):
    query_result = google_places.nearby_search(
        lat_lng={'lat': lat, 'lng': lng}, 
        radius=radius,
        types=[types.TYPE_HOSPITAL])
    return query_result

def extractData(queryRst,lat, lng):
    Rst = pd.DataFrame(index = range(1000), columns = ['HospitalName', 'Latitude', 'Longitude','DistanceInMiles'])
    for i in range(len(queryRst.places)):
        Rst['HospitalName'][i] = queryRst.places[i].name
        Rst['Latitude'][i] = queryRst.places[i].geo_location['lat']
        Rst['Longitude'][i] = queryRst.places[i].geo_location['lng']
        Rst['DistanceInMiles'][i] = geopy.distance.vincenty((lat, lng), (Rst['Latitude'][i], Rst['Longitude'][i])).miles

    Rst = Rst.dropna(how='all')  
    return Rst
    

In [49]:
stp = 0.1
startLat = 31.99
startLng = -106.61

dfList = []

for la in range(5):
    for lg in range(5):
        lat = round(startLat - stp * la, 2)
        lng = round(startLng + stp * lg, 2)
        queryRst = queryGooglePlace(lat, lng, 7285)
        Rst = extractData(queryRst,lat, lng)
        dfList.append(Rst)
    
        

In [50]:
len(dfList)

25

In [51]:
fnl = pd.concat(dfList, axis=0)
len(fnl)

216

In [59]:
fnl.columns

Index(['HospitalName', 'Latitude', 'Longitude', 'DistanceInMiles'], dtype='object')

In [60]:
fnl1  = fnl[['HospitalName', 'Latitude', 'Longitude']].drop_duplicates()
len(fnl1)

169

In [61]:
fnl1.to_excel('ElPasoHospitals.xlsx', index=False)